# Face to Cartoon with CycleGAN

## Advanced Topics of Machine Learning (FS2019)

##### Guodong Zeng, Benjamin Fankhauser, Jan Segessenmann, Gautam Ilango

### Introduction 

Our goal is to switch image styles from real faces to cartoon faces using unpaired data only. Some of the important features of the real faces should be preserved and recognized in the generated cartoon faces.  
As a starting point, we use an implementation of [CycleGAN](http://openaccess.thecvf.com/content_ICCV_2017/papers/Zhu_Unpaired_Image-To-Image_Translation_ICCV_2017_paper.pdf) introduced by Jun-Yan Zhu et al. CycleGAN achieves impressive results in learning the mapping between different styles of images (e.g. paintings to photos, zebras to horses or summer to winter and vice versa). As there is quite some successful research done in image to image translation, one can also find different approaches to achieve our specific goal in literature (mostly by turning real faces into caricatures, like one can see it from [Microsoft's AI](https://www.zdnet.com/article/how-microsofts-caricature-ai-turns-photos-into-portrait-cartoons/)). Our goal however was to get some insight by ourselfes on how to use and improve CycleGAN for a given task like this. 



Improvement was mainly done by two additional loss terms. In order to incorporate prior knowledge to the architecture we add another loss term based on landmark predictions for both, the real faces and the cartoon faces.
To extend the usages of our model we additionally provide a one to ten mapping of real faces to cartoon faces. We make the whole architecutre conditional and train the models with ten different hair colors, so the user is free to choose his or her hair color.

In this report we describe the materials and methods that were used before examining our first results without any additional improvements on the algorithms. We will afterwarts explain these encancements and the resulting image to image translation. There is also a runnable demonstration (the code is found external to this report). We close with a short conclusion and outlook in the last section.


### Materials and Methods
This chapter briefly covers our methods and the materials (e.g. already existing algorithms we made use of) underlying our work are introduced on to the reader. Even though we can expect CycleGAN to do well on our task, we tried DCGAN as a fist approach to widen our horizon on methods which seem promising in generating cartoons, before narrowing our sight to CycleGAN.
#### DCGAN
As we searched for different methods to achieve our task, we used DCGAN to gain some insight in cartoon generation and its possible problem fields. A Deep Convolution Generative Adversarial Network learns the mapping from gaussian noise to the targeted domain using a generative loss. The power of the deep deconvolution layout has shown its success in various examples where artificial Images needs to be generated.
We use this method to compare our results and use the code provided by [pytorch/dcgan](https://github.com/pytorch/examples/tree/master/dcgan).

The DCGAN leads to the image below on the cartoon dataset. The generator has learnt to combine various features of the cartoons. But the results are not acceptable: deeper structures like equal skin color have not been learnt. The generator generates women with beards. But we can not blame him as the cartoon dataset contains women with beards too. The input of the generator is gaussian noise and not real face images. So it does not solve our task.

![dcgan](doc/images/dcgan-fake-sample.png) 

#### CycleGAN
To achieve the unpaired style transfer from real faces to cartoon faces, we have to learn the mapping function $G: A \to B$, with domain $A$ being real faces and domain $B$ being cartoon faces, such that the distribution $G(A)$ is as similar as possible to the distribution $B$ by minimizing adversarial loss. Since this mapping is highly under-constrained, Jun-Yan Zhu et al. introduced the coupling of another mapping $H: B \to A$. The main idea is to minimize the consistency cycle losses $L(A, (H(G(A)))$ and $L(B, (G(H(B)))$ as well as the usual adversarial losses.  
![scheme](doc/images/cycleGAN_scheme_extended.png)  
The above figure shows schematically the workflow of CycleGAN with its loss terms. As a nice bonus, we will also be able to transfer cartoons into real faces. The terms real and fake as you can see them in the figure are used to declare if the corresponding image is an original or has been generated and should not be confused with real faces as we use it throughout this report do describe images of real people.

The paper on [CycleGAN](https://junyanz.github.io/CycleGAN/) promises a more restricted training procedure for better results than we could get with DCGAN.
The first results are acceptable. But we discover two problems: 
- Mode collapse: the generator only generates a few modes of the original cartoon dataset. 
- Liveliness: the cartoons look exactly like the cartoon data, but we want something more human like: Humans turn and move their head and eyes. We would like to incorporate this feature to the generated cartoons to have more correspondence between the input face and the generated cartoon.

![first result of cyclegan](doc/images/cyclegan-first.jpg)

In the above figure one sees an input image to the left and generated fake image by CycleGAN to the right.

Based on the first results with CycleGAN, we decided that this method is worth to investigate more. We propose two approaches to solve the described problems: to reduce mode collapse we will build a conditional GAN architecture to the generator network. To encounter the liveliness we introduce a new loss term described in the chapter Landmark Loss in CycleGAN.
#### Datasets
We searched for different datasets on real images of faces and found some shared data listed in [Links.md](https://github.com/fs2019-atml/face-to-cartoon/blob/master/Links.md). A Link to all datasets (inluding the cartoon images) is given there. Important criteria for our purpose were (1) as little background (its considered noise in our case) as possible, (2) as much colored images as possible, (3) iid data to represent the real distribution of human faces as good as possible, (4) frontal perspective, as we find it in the cartoon set. We selected 1000 real face images from the FFHQ dataset, which is from [Style GAN](https://arxiv.org/pdf/1812.04948.pdf) and contains some nice images which should serve our purpose well. Other datasets like labeled faces in the wild were also used, but we considered them too noisy.

We took the cartoon images from Goolge. These cartoons helped develop the technology behind the personalized stickers in Google Allo and are free to download and use. The set is generated by different combinations of a few parameters like hair style, color, glasses, etc. There are about $10^13$ possible combinations and they are random, thus we also find combinations which are very rare to find in a real-world scenario and therefore the feature distribution is not very similar to the real face images. Nevertheless there are enough similarities to expect good results. Below, you see some examples of cartoon images. Some features like e.g. the mouth remain the same throughout the dataset.
![cartoonset](doc/images/cartoonset.PNG)

#### Landmark labeling
Before introducing a landmark loss as described below, we had to label the real faces. This was done by a short matlab script, were one can set all the landmarks manually. To guarantee that the landmarks are well positioned, one can verified every landmark by eye with yet another script. The scripts can be found in [here](https://github.com/fs2019-atml/face-to-cartoon/tree/master/code/landmarks).




### Landmark Loss in CycleGAN
To incorporate more liveliness to the generated cartoons, the goal was to create some kind of correspondence among the real faces and the generated cartoons. As both domains are faces we enforce correspondence on their landmarks: a fake image should preserve the landmarks of the original image.

To incorporate this prior knowlege about the domains we introduce the landmark (LD) loss. The prior knowledge about the real faces should be preserved through the generator:

![CycleGAN with Landmarkloss](doc/images/cyclegan-ldnet.png)

The landmark loss predicts the landmarks on each domain and compares the prediction on the generated image to the ground truth on the input. This process requires more labeling but the faces and cartoons are still unpaired.

#### Implementation
The Landmark loss is a simple convolutional network with five convolutional layers and two for the final regression. We call it LDNet and it has been inspired by literature about landmark detection networks in general.

LDNet outputs five cooridnates for five landmarks. The final loss is then computed with a mean squared error between the predicted and given landmarks.

![ldnet](doc/images/ldnet.png)

The five landmarks are: 1) left eye, 2) right eye, 3) nose, 4) left mouth, 5) right mouth.

#### Training and dataset
Like the discriminator we train after each generator pass. We have two instances of the network: the landmark detector for real faces and the landmark detector for cartoon faces. We train both with the real image given by the dataset. We use only one sample to train and convergence is slow. In general this is not a problem as CycleGAN requires a lot of training and the landmark detection networks convergence is quick compared to the huge generator networks.

For the real faces we labeled 1000 images by hand. The cartoon faces have their 5 landmarks aligned at the same position. So we got the cartoon landmarks for free. But to train a meaningful LDNet (static position would be too easy to learn) we had to implement a random crop in such a way that the landmarks are still
correct after croping. We refer to our own [face_dataset.py](https://github.com/fs2019-atml/face-to-cartoon/blob/master/code/data/face_dataset.py) implementation.

During training time the generator gets his gradients from the discriminator, the cycle and the landmark detection. It is crucial to find the right balance between the loss terms. The landmark loss can produce a state where it is too easy for the discriminator to separate real from fake images. Experiments have shown that a small landmark lambda of 0.01 to 0.0001 introduces landmark correspondences without destroying the overall learning (the discriminator has a lambda of 1 and the cycle a lambda of 10).


#### Results

Using a lambda of 0.01 for the landmark loss we see a correspondence in the generated fakes:

![landmark correspondences](doc/images/landmark-correspondences.png)


It is hard to tell what exactly the impact of the landmark loss on the generator is. But we can show what we generate with and without landmark loss.

![landmark vs original](doc/images/landmark-vs-original.png)

The first column contains a test image and its corresponding fake is the second column. The fake is created by a landmark loss trained generator. The fake does not exactly look like the original cartoon which is in the third column. The landmark fakes cover some kind of facial expression as in the real image (e.g. viewing direction, perspective scaling of the right eye of the girl in the bottom row). In the fourth column we have fakes without landmark loss, they capture more of the original distribution (e.g. the unnatural, direct look to the camera).

#### Limitations

Depending on the task we provide a lambda where you can control the amount of correspondence. A even lower landmark lambda leads to less correspondence until you can not determine if there was a landmark loss or not. A lamda too high on the other hand will destroy the learning process: the distortions will be too much to accept them as part of the target distribution. In this case the discriminator can easily distinguish between real and fakes and the generator is unable to find a solution satisfying both losses.

This induced distortion is the limitation of the approach: the trainig process is less stable and takes longer to converge.







### Conditions on Hair Color

After the training of CycleGAN, we only get one generator function from face image input to cartoon image output. We have no control of the cartoon image generation. For our task it would be preferable to have control over features like the hair color, glasses, gender etc. Based on above observation, we hope to control the hair color of cartoon image generation with the following approach.

#### Conditional GAN


[Conditional GAN ](https://arxiv.org/pdf/1411.1784.pdf) shows that the model can generate MNIST digits conditioned on class labels. The main idea is to use a class label to condition on to both the generator and discriminator, which is illustrated as below: 
![alt text](https://www.researchgate.net/profile/Alptekin_Temizel/publication/326928177/figure/fig4/AS:736665580609547@1552646169957/a-Standard-GAN-and-b-conditional-GAN-architectures.ppm "Logo Title Text 1")



Instead of taking only real face image as input we added additional class label input for hair color to the generator and discriminator of CycleGAN. Given a real face image, we can generate cartoon images for different hair color with different input of hair color class labels. 

To do so we implemented a one-hot encoding and reapply this mask in the first few layers of the resnet generator architecture.

#### Training Dataset
We have two modalities of images, i.e. real face image dataset (1000 images) and cartoon face images. For cartoon face images we have 10 different hair colors and for each hair color we have 200 images. In total we use 2,000 cartoon face images for training. The training of the model is in an unpaired way. 


#### Results

Given one real face image input, we can generate 10 cartoon images with different hair color. 
![alt text](https://github.com/fs2019-atml/face-to-cartoon/raw/master/doc/images/ConditionalOutput.png "Logo Title Text 1")


The model does not perform so well on test image with glasses. The reason may be that there ia only a small number of real face images with galsses in the training set. 
![alt text](https://github.com/fs2019-atml/face-to-cartoon/raw/master/doc/images/GlassResult.png "Logo Title Text 1")


### Final Architecture and demonstration
Landmark loss and conditional architecure have been developed in parallel. Finally we merged the code and datasets and trained the network again. We run the final training four times with four different lambdas for the landmark loss. Using the latest configuration one full training requires around 10 days on the cluster. A few results are presented below.
For demonstration purposes, the webcamera is used to capture a picture and feed it through the generator network. The latest trained model weights are applied to a forward pass. The additional class label is used to apply the condition on hair color of the cartoon image as described above.
You can find and run our interactive demo (requires a webcam) [here](https://github.com/fs2019-atml/face-to-cartoon/blob/master/code/Demo.ipynb)

![With glasses](doc/images/demo1.png)
![Without glasses](doc/images/demo2.png)


### Conclusion and Outlook
As one can see in the final results, we managed to translate real faces to cartoon faces, preserving some important features (e.g. beards, glasses, some important landmarks etc.) for recognition and thus achieved our inital goal quite well. We also showed that with the landmark loss, CycleGAN can be extended in a natural way. The provided method has limitations and we assume that there are better methods around e.g. for aligning the landmarks, but most of those methods might be much more complex than our straight forward extension. With our general approach we should be able to constraint any little feature correspondence through the generator.

Building each component was not too hard of a task. The main problem was to take the right decisions after analizing the output. We had different face datasets and hyperparameters to tune. But a small trainig session for meaningfull results required 24 to 36 hours of training on the provided cluster. And each result had to be analyzed and discussed to determine if our expectation have met or if it was just a random occurance. Our group was lucky to have access to another machine with four Teslas V100 (each is about 4-5 times faster per iteration compared to the cluster which reduced the 24 to 6 hours) and yet another machine with two GPUs. But each additional machine introduced some complexity in handling and exchanging data and results. Even with 7 GPUs in use we could not perform every test and still had to relay on intuition.

To improve the model further, one could set some more landmarks to preserve the shapes of e.g. the ears or the hair etc. Also, the algorithm is quite sensitive to background and illumination as well as view point. Another way to enhance the performence was to identify background in the real images and set it to white to have nice contours. Trough more augmentation of the dataset (e.g. rotation on the cartoon images) one might also achieve better results in viewpoint and illumination. The conditional architecture can be extended such that e.g. the skin color is also considered. This problem is described and implemented with a different approach as stated in this [paper](https://research.fb.com/wp-content/uploads/2017/08/unsupervised-creation-parameterized.pdf).

## Appendix

### List of contributions
In case it is needed for evaluation, we provide a short table on the tasks and the main contributers per task, since the number of merge commits or changed lines alone are not really representative. We tried to split the tasks according to our foreknowledges and experiences for effectivity of our project as well as on personal learning effect.

| Task                           	| Main contributors              	|
|--------------------------------	|--------------------------------	|
| Project meetings and planning  	| Everyone                       	|
| Search and provide datasets    	| Jan, Gautam                    	|
| Landmark labeling 1000 faces   	| Jan                            	|
| Landmark loss                  	| Benjamin                       	|
| Conditional CycleGAN           	| Guodong                        	|
| Training of final model        	| Benjamin, Guodong              	|
| Report                         	| Everyone                       	|
| Presentation                   	| Everyone                       	|
| Demo                           	| Jan, Benjamin, Gautam          	|